<a href="https://colab.research.google.com/github/pcbzmani/AV_Jobathon_Sep_2022/blob/main/AV_Nov2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [89]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("energypredication")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [90]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

In [91]:
ls drive/MyDrive/Jobathon_Nov2022/

res/  sample_submission_jn0a7vR.csv  test_WudNWDM.csv  train_IxoE5JN.csv


In [92]:
train_data_df = spark.read.csv('/content/drive/MyDrive/Jobathon_Nov2022/train_IxoE5JN.csv',header=True,inferSchema=True)

In [45]:
train_data_df = train_data_df.withColumn('month', F.split(F.col('datetime'), '-')[1])\
                             .withColumn('day', F.split((F.split(F.col('datetime'), '-')[2]),' ')[0])\
                             .withColumn('time', F.split((F.split(F.col('datetime'), ' ')[1]),':')[0])\
                             .withColumn('daymonthtime',F.concat(F.col('day'),F.col('month'),F.col('time')).cast('int'))\
                             .drop('month','day','time')\
                             .dropna()

In [46]:
train_data_df.show()

+------+-------------------+------------------+------------+
|row_id|           datetime|            energy|daymonthtime|
+------+-------------------+------------------+------------+
|     1|2008-03-01 00:00:00|      1259.9855625|       10300|
|     2|2008-03-01 01:00:00|         1095.5415|       10301|
|     3|2008-03-01 02:00:00|         1056.2475|       10302|
|     4|2008-03-01 03:00:00|          1034.742|       10303|
|     5|2008-03-01 04:00:00|         1026.3345|       10304|
|     6|2008-03-01 05:00:00|         1033.7685|       10305|
|     7|2008-03-01 06:00:00|           1086.78|       10306|
|     8|2008-03-01 07:00:00|          1211.742|       10307|
|     9|2008-03-01 08:00:00|          1293.693|       10308|
|    10|2008-03-01 09:00:00|1318.9154999999998|       10309|
|    11|2008-03-01 10:00:00|          1337.943|       10310|
|    12|2008-03-01 11:00:00|          1348.917|       10311|
|    13|2008-03-01 12:00:00|         1360.6875|       10312|
|    14|2008-03-01 13:00

In [47]:
assembler_obj = VectorAssembler(inputCols=['daymonthtime'],outputCol='daymonthtime_feature')

In [48]:
feature_vector_df = assembler_obj.transform(train_data_df)

In [49]:
formatted_data = feature_vector_df.select('daymonthtime_feature','energy')

In [50]:
train_data, test_data = formatted_data.randomSplit([0.7,0.3])

In [51]:
lireg = LinearRegression(featuresCol='daymonthtime_feature', labelCol= 'energy')
lireg_model = lireg.fit(train_data)
test_results = lireg_model.evaluate(test_data)

In [52]:
test_results.rootMeanSquaredError

369.61808761634614

In [53]:
test_results.r2

-0.00017967367921323252

In [54]:
unlabeled_data = test_data.select('daymonthtime_feature')
predictions = lireg_model.transform(unlabeled_data)

In [55]:
predictions.show()

+--------------------+------------------+
|daymonthtime_feature|        prediction|
+--------------------+------------------+
|           [10100.0]|1702.2860094187909|
|           [10100.0]|1702.2860094187909|
|           [10100.0]|1702.2860094187909|
|           [10100.0]|1702.2860094187909|
|           [10101.0]|1702.2860251151035|
|           [10101.0]|1702.2860251151035|
|           [10102.0]|1702.2860408114161|
|           [10102.0]|1702.2860408114161|
|           [10102.0]|1702.2860408114161|
|           [10103.0]|1702.2860565077287|
|           [10103.0]|1702.2860565077287|
|           [10103.0]|1702.2860565077287|
|           [10103.0]|1702.2860565077287|
|           [10104.0]|1702.2860722040414|
|           [10104.0]|1702.2860722040414|
|           [10104.0]|1702.2860722040414|
|           [10105.0]| 1702.286087900354|
|           [10105.0]| 1702.286087900354|
|           [10105.0]| 1702.286087900354|
|           [10105.0]| 1702.286087900354|
+--------------------+------------

In [56]:
energy_coeff = lireg_model.coefficients[0]

In [57]:
intercept = lireg_model.intercept

In [58]:
energy_coeff

1.5696312609903523e-05

In [59]:
intercept

1702.1274766614308

In [60]:
predict = (intercept) + (energy_coeff) * (10100)

In [61]:
predict

1702.2860094187909

In [63]:
test_data_df = spark.read.csv('/content/drive/MyDrive/Jobathon_Nov2022/test_WudNWDM.csv',header=True,inferSchema=True)

In [64]:
test_data_df = test_data_df.withColumn('month', F.split(F.col('datetime'), '-')[1])\
                             .withColumn('day', F.split((F.split(F.col('datetime'), '-')[2]),' ')[0])\
                             .withColumn('time', F.split((F.split(F.col('datetime'), ' ')[1]),':')[0])\
                             .withColumn('daymonthtime',F.concat(F.col('day'),F.col('month'),F.col('time')).cast('int'))\
                             .drop('month','day','time')

In [82]:
def enrgycal(inpcol):
  predict = (1702.1274766614308) + (1.5696312609903523e-05) * (inpcol)
  return predict

In [83]:
convertUDF = F.udf(lambda z: enrgycal(z))

In [84]:
test_data_df = test_data_df.withColumn('energy', convertUDF(F.col('daymonthtime')))

In [86]:
res = test_data_df.select('row_id','energy')

In [87]:
res.repartition(1).write.format('csv').option('header',True).save('/content/drive/MyDrive/Jobathon_Nov2022/res')